In [4]:
def recalc_pyroxene(path,sheet_name):
    """
    READ ME
    
    This will recalculate olivine analysis and propagate errors with a standard error analysis for Fs, En, Di, and Fh
    AlIV and AlIII are calculated by difference
    Fe2+ and Fe3+ are calculated based on an O2- deficit
    
    Inputs:
        path = some path name to your file. This must be input in parentheses and end in .xlsx
        Your file headers must be in all lower case. For instance, SiO2 must be sio2, etc
        Your first column must be the name that you gave your spot when measuring
        sheet_name = input the sheet name of that excel file where your data lives. This must also be in parentheses. 
        
    """
    #import dependables
    import pandas as pd
    import numpy as np
    import math
    from statistics import stdev
    df = pd.read_excel(path,sheet_name)
    #set up a general dataframe that has a number of oxides, their number of O2-, the cation:anion, and the molecular weights
    oxides = pd.DataFrame([[60.0855,101.963,94.1966,56.087,79.867,70.938,71.845,61.9796,40.305,141.9476,151.990,74.693,64.066,153.326,149.881,265.81,81.379,18.998,35.453,80.063],
                          [2,3,1,1,2,1,1,1,1,5,3,1,2,1,3,5,1,1,1,3],
                          [1/2,2/3,2,1,1/2,1,1,2,1,2/5,2/3,1,1/2,1,2/3,2/5,1,1,1,1/3]],
                      columns=['sio2','al2o3','k2o','cao','tio2','mno','feo','na2o','mgo','p2o5','cr2o3','nio','so2','bao','v2o3','nb2o5','zno','f','cl','so3'])
    #pull out the column with strings
    spots = df.iloc[:,0]
    #pull out all the columns with data in them
    data = df.iloc[:,1:]
    #set any negative numbers = 0
    data[data < 0] = 0
    #put together the measured data and the general oxides dataframe, dropping any columns from 'oxides' that does not exist in the measured dataset 
    newframe = pd.concat([data,oxides], join='inner')

    #the line below does a few things:
    #1. gives you the molar proportion of oxides by dividing measurements by molecular weights
    #2. reorganizes the dataframes to retain the oxides dataframe in a neat manner
    #3. adds a suffix for the eventual output dataframe accordingly
    newframe = pd.concat([newframe.iloc[:-3,:].div(newframe.iloc[-3,:]),newframe.iloc[-3:]]).reset_index().drop('index',axis=1).add_suffix('_molar proportion')

    #multiply by the number of oxygens per oxide in order to get the atomic proportion of oxygen
    atomico2 = newframe.iloc[:-3,:].multiply(newframe.iloc[-2,:])
    #add up the atomic proportion of oxygen. This helps you normalize your measurement in a step below
    atomico2['number o2-'] = atomico2.sum(axis=1)
    #make a normalizing factor for each analysis based on an ideal number of oxygen with respect to what you actually measured
    atomico2['normalizing factor'] = 6 / atomico2['number o2-']
    #put the oxides information back in for ease of calculating and cleanliness
    atomico2 = pd.concat([atomico2,newframe.iloc[-3:]])

    #get the amount of negative charge that is required for each of the cations you measured. This is effectively how your ideal number of negative charges (which you've obtained by normalizing) are distributed about your mineral site based on the cations that are actually measureable.
    anions = atomico2.iloc[:-3,:-2].multiply(atomico2.iloc[:-3,-1],axis=0)
    #sum the total number of negative charges. This should sum up to the numerator you have used above. For the case of olivine, this would be 4.
    anions['negative charge total'] = anions.sum(axis=1)
    #put the oxides information back in for ease of calculating and cleanliness
    anions = pd.concat([anions,newframe.iloc[-3:]])
    #change the suffix at the end of this series of columns for outputting later
    anions.columns = anions.columns.str.replace('_molar proportion','_charge distribution')

    #multiply the distributed oxygen charge by the ratio of cation to anion. A simple unit analysis should guide your intution
    cations = anions.iloc[:-3,:-1].multiply(anions.iloc[-1,:-1])
    #sum the number of cations. This should be pretty close to your mineral stoichiometry.
    cations['sum cations'] = cations.sum(axis=1)
    #change the suffix for hte purposes of outputting later
    cations.columns = cations.columns.str.replace('_charge distribution','_cations unnormalized')
    #calculate the normalization needed to get an ideal number of cations. For pyroxenes this is done to get Fe2+ and Fe3+. There are some critical assumptions here and you should always keep the limitations of your measurements in mind
    cations['cation norm factor'] = 4 / cations['sum cations']

    #get the normalized number of cations, based on an ideal number of 4
    cations_norm = cations.iloc[:,:].multiply(cations.iloc[:,-1],axis=0)
    cations_norm.columns = cations_norm.columns.str.replace('_cations unnormalized','_cations normalized')
    #sum them. This should equal 4 for pyroxenes
    cations_norm['sum normalized cations'] = cations_norm.sum(axis=1)

    anions_norm = anions
    anions_norm['cations norm factor'] = 4 / cations['sum cations']
    anions_norm = anions_norm.iloc[:-3,:-2].multiply(anions_norm.iloc[:-3,-1],axis=0)
    anions_norm['sum normalized anions'] = anions_norm.sum(axis=1)
    anions_norm['Needed O2'] = 6 - anions_norm['sum normalized anions']

    cations_final = cations_norm.drop(['al2o3_cations normalized','feo_cations normalized','sum cations','cation norm factor','sum normalized cations'],axis=1)
    cations_final['Fe3+'] = [x*2 if x>0 else 0 for x in anions_norm['Needed O2']]
    cations_final['Fe2+'] = [i-x if x>0 else i for x,i in zip(cations_final['Fe3+'],anions_norm['feo_charge distribution'])]
    cations_final['Al(IV)'] = [i if 2-x>i else 2-x for i,x in zip(cations['al2o3_cations unnormalized'],cations['sio2_cations unnormalized'])]
    cations_final['Al(III)'] = cations['al2o3_cations unnormalized'] - cations_final['Al(IV)']
    cations_final.columns = cations_final.columns.str.replace('_cations normalized','_as cation')
    if "na2o_as cation" in cations_final is not False:
        cations_final['na2o_as cation'] = cations_final['na2o_as cation']
    else:
        cations_final['na2o_as cation'] = 0

    cations_final['Endmembers Total'] = cations_final['na2o_as cation'] + cations_final['mgo_as cation'] + cations_final['Fe3+'] + cations_final['Fe2+'] + cations_final['cao_as cation'] + cations_final['mno_as cation']
    cations_final['En'] = cations_final['mgo_as cation'] / cations_final['Endmembers Total']
    cations_final['Wo'] = cations_final['cao_as cation'] / cations_final['Endmembers Total']
    cations_final['Fs'] = (cations_final['Fe3+']+cations_final['Fe2+']) / cations_final['Endmembers Total']

    cations_final['En error propagated'] = cations_final['En'].mean() * ((stdev(cations_final['mgo_as cation'])/(cations_final['mgo_as cation'].mean()))**2 + (((stdev(cations_final['cao_as cation']))**2+(stdev(cations_final['mgo_as cation'])**2)+(stdev(cations_final['Fe3+'])**2)+(stdev(cations_final['Fe2+'])**2)+(stdev(cations_final['mno_as cation'])**2)+(stdev(cations_final['na2o_as cation'])**2))**(1/2)/(cations_final['Endmembers Total'].mean()))**2)**(1/2)
    cations_final['Wo error propagated'] = cations_final['Wo'].mean() * ((stdev(cations_final['cao_as cation'])/(cations_final['cao_as cation'].mean()))**2 + (((stdev(cations_final['cao_as cation']))**2+(stdev(cations_final['mgo_as cation'])**2)+(stdev(cations_final['Fe3+'])**2)+(stdev(cations_final['Fe2+'])**2)+(stdev(cations_final['mno_as cation'])**2)+(stdev(cations_final['na2o_as cation'])**2))**(1/2)/(cations_final['Endmembers Total'].mean()))**2)**(1/2)
    cations_final['Fs error propagated'] = cations_final['Fs'].mean() * (((stdev(cations_final['Fe3+'])**2+stdev(cations_final['Fe2+'])**2)**(1/2)/((cations_final['Fe3+'].mean())+cations_final['Fe2+']))**2 + (((stdev(cations_final['cao_as cation']))**2+(stdev(cations_final['mgo_as cation'])**2)+(stdev(cations_final['Fe3+'])**2)+(stdev(cations_final['Fe2+'])**2)+(stdev(cations_final['mno_as cation'])**2)+(stdev(cations_final['na2o_as cation'])**2))**(1/2)/(cations_final['Endmembers Total'].mean()))**2)**(1/2)

    pyroxene_recalc = pd.concat([spots,data,atomico2,anions,cations,cations_norm,anions_norm,cations_final],axis=1).fillna(0)
    output = pyroxene_recalc.to_excel('Pyroxene_recalculated.xlsx')

    return output

In [7]:
path = '/Users/ctlewis/Library/Mobile Documents/com~apple~CloudDocs/Documents/Projects/Caspana_052921/Phases/PyxCH12020_2.xlsx'
sheet_name = 'py_code'
recalc_pyroxene(path,sheet_name)